<a href="https://colab.research.google.com/github/bonnefco/P8/blob/main/P8_FIRST_CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation des librairies

In [ ]:
from google.colab import drive

drive.mount("/content/drive")
!pip install black[jupyter]
!black /content/drive/MyDrive/Colab_Notebooks/P8/P8-main/P8_FIRST_CLEANING.ipynb

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
import ast
from scipy import stats

# Un apreçu du dataset

In [ ]:
df_effective_args = pd.read_csv(
    "/content/drive/MyDrive/Colab_Notebooks/P8/dataset/train.csv",
    encoding="utf-8",
    header=(0),
)

In [ ]:
df_effective_args

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective


# Nettoyage

## Recherche et suppression des doublons

In [ ]:
df_duplicates = df_effective_args[
    df_effective_args.duplicated(subset=["discourse_text"], keep=False)
]

### Est ce que des doublons possèdent deux "discourse_effectiveness" différents ?

In [ ]:
df_duplicates_different_discourse_effectiveness = df_duplicates[
    ~df_duplicates.duplicated(
        subset=["discourse_text", "discourse_effectiveness"], keep=False
    )
]

In [ ]:
df_duplicates_different_discourse_effectiveness.shape

(22, 5)

In [ ]:
list_index_to_drop = (
    df_duplicates_different_discourse_effectiveness.index.values.tolist()
)
len(list_index_to_drop)

22

### Est ce que des doublons possèdent deux "discourse_type" différents ?

In [ ]:
df_duplicates_different_discourse_type = df_duplicates[
    ~df_duplicates.duplicated(subset=["discourse_text", "discourse_type"], keep=False)
]

In [ ]:
list_index_to_drop += df_duplicates_different_discourse_type.index.values.tolist()
len(list(set(list_index_to_drop)))

30

#### Il y a 30 lignes possédant des incohérences

### Conservation des doublons dont les "variable discourse_type" et 	"discourse_effectiveness" sont homogènes

In [ ]:
df_without_incoherence = df_effective_args.drop(list_index_to_drop, axis=0)

### Suppression des derniers doublons

In [ ]:
df_effective_args_without_duplicates = df_without_incoherence.drop_duplicates(
    subset=["discourse_text"], keep="first"
)

In [ ]:
df_effective_args_without_duplicates

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective


## Suppression des variables jugees inutiles
* Justification : les appreciations sont indépendantes 

In [ ]:
df_effective_args_less_variables = df_effective_args_without_duplicates[
    ["discourse_text", "discourse_type", "discourse_effectiveness"]
]
df_effective_args_less_variables.head(2)

,discourse_text,discourse_type,discourse_effectiveness
0,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,"On my perspective, I think that the face is a ...",Position,Adequate


# Sauvegarde du fichier

In [ ]:
df_effective_args_less_variables.to_csv(
    "/content/drive/MyDrive/Colab_Notebooks/P8/dataset/train_light.csv", index=False
)